In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import pandas as pd
import time

In [6]:
reserve = pd.read_csv("filtered_reviews_.csv")
cafe = pd.read_csv("cafe_con_leche_reviews_filtered.csv")
gwen = pd.read_csv("upstairs_gwen_reviews_filtered.csv")
weather = pd.read_csv("weather_data.csv")

In [8]:
print(reserve.columns)

Index(['Date', 'Rating'], dtype='object')


In [10]:
print(weather.columns)

Index(['City', 'Date', 'Temp Max (°F)', 'Temp Min (°F)', 'Avg Temp (°F)',
       'Feels Like Temp (°F)', 'Humidity (%)', 'Precipitation (in)',
       'Snow (in)', 'Snow Depth (in)', 'Wind Speed (mph)', 'Cloud Cover (%)',
       'Visibility (mi)', 'UV Index', 'Severe Weather Risk', 'Conditions'],
      dtype='object')


In [12]:
print(cafe.columns)

Index(['Date', 'Rating'], dtype='object')


In [14]:
print(gwen.columns)

Index(['Date', 'Rating'], dtype='object')


In [16]:
# Load reviews
reserve = pd.read_csv("filtered_reviews_.csv")
# Clean and convert date
reserve["Clean_Date"] = reserve["Date"].str.replace("Dined on ", "").str.strip()
reserve["Clean_Date"] = pd.to_datetime(reserve["Clean_Date"], format="%B %d, %Y")
reserve["Location"] = "Starbucks Reserve"  
reserve["NumericRating"] = reserve["Rating"].str.extract(r"(\d)").astype(float)

cafe = pd.read_csv("cafe_con_leche_reviews_filtered.csv")
cafe["Clean_Date"] = cafe["Date"].str.replace("Dined on ", "").str.strip()
cafe["Clean_Date"] = pd.to_datetime(cafe["Clean_Date"], format="%B %d, %Y")
cafe["Location"] = "Café Con Leche" 
cafe["NumericRating"] = cafe["Rating"].str.extract(r"(\d)").astype(float)

gwen = pd.read_csv("upstairs_gwen_reviews_filtered.csv")
gwen["Clean_Date"] = gwen["Date"].str.replace("Dined on ", "").str.strip()
gwen["Clean_Date"] = pd.to_datetime(gwen["Clean_Date"], format="%B %d, %Y")
gwen["Location"] = "Upstairs at the Gwen"
gwen["NumericRating"] = gwen["Rating"].str.extract(r"(\d)").astype(float)

In [18]:
reviews = pd.concat([reserve, cafe, gwen], ignore_index=True)
reviews = reviews.sort_values("Clean_Date", ascending=False).reset_index(drop=True)

In [20]:
print(reviews["Location"].value_counts())

Location
Upstairs at the Gwen    87
Café Con Leche          56
Starbucks Reserve       28
Name: count, dtype: int64


In [22]:
display(reviews)

,Date,Rating,Clean_Date,Location,NumericRating
0,"Dined on March 29, 2025",4.3 stars,2025-03-29,Starbucks Reserve,4.0
1,"Dined on March 27, 2025",4.3 stars,2025-03-27,Starbucks Reserve,4.0
2,"Dined on March 27, 2025",3 stars,2025-03-27,Upstairs at the Gwen,3.0
3,"Dined on March 24, 2025",4.1 stars,2025-03-24,Starbucks Reserve,4.0
4,"Dined on March 22, 2025",4.6 stars,2025-03-22,Café Con Leche,4.0
...,...,...,...,...,...
166,"Dined on May 5, 2023",3.6 stars,2023-05-05,Café Con Leche,3.0
167,"Dined on May 5, 2023",4.7 stars,2023-05-05,Café Con Leche,4.0
168,"Dined on April 22, 2023",4.8 stars,2023-04-22,Café Con Leche,4.0
169,"Dined on April 14, 2023",4.4 stars,2023-04-14,Café Con Leche,4.0


In [24]:
# Load weather data
weather = pd.read_csv("weather_data.csv")
weather["Date"] = pd.to_datetime(weather["Date"], format="%Y-%m-%d")  

In [26]:
merged = pd.merge(
    reviews,
    weather,
    left_on="Clean_Date",
    right_on="Date",
    how="left"
)

In [28]:
merged = merged.drop(columns=["Date_x"])

In [30]:
print(merged.head(20))

       Rating Clean_Date              Location  NumericRating  \
0   4.3 stars 2025-03-29     Starbucks Reserve            4.0   
1   4.3 stars 2025-03-27     Starbucks Reserve            4.0   
2     3 stars 2025-03-27  Upstairs at the Gwen            3.0   
3   4.1 stars 2025-03-24     Starbucks Reserve            4.0   
4   4.6 stars 2025-03-22        Café Con Leche            4.0   
5   4.8 stars 2025-03-20     Starbucks Reserve            4.0   
6   4.3 stars 2025-03-16     Starbucks Reserve            4.0   
7     5 stars 2025-03-15  Upstairs at the Gwen            5.0   
8   4.8 stars 2025-03-15     Starbucks Reserve            4.0   
9   4.1 stars 2025-03-14     Starbucks Reserve            4.0   
10  3.6 stars 2025-03-14        Café Con Leche            3.0   
11  4.8 stars 2025-03-09     Starbucks Reserve            4.0   
12    2 stars 2025-03-09  Upstairs at the Gwen            2.0   
13    4 stars 2025-03-08  Upstairs at the Gwen            4.0   
14    5 stars 2025-03-08 

In [28]:
merged.to_csv("reviews_with_weather.csv", index=False)

In [32]:
column_names = [
    "Rating", "Clean_Date", "Location", "NumericRating", "City",
    "Date_y", "Temp Max (°F)", "Temp Min (°F)", "Avg Temp (°F)", "Feels Like (°F)",
    "Humidity (%)", "Precipitation (in)", "Snow (in)", "Snow Depth (in)",
    "Wind Speed (mph)", "Cloud Cover (%)", "Visibility (mi)", "UV Index",
    "Severe Weather Risk", "Conditions"
]

descriptions = [
    "Star rating in text form (e.g., '4.3 stars')",
    "Cleaned review date in YYYY-MM-DD format",
    "Restaurant or cafe name where review was left",
    "Numeric version of star rating (float)",
    "City and state of the location (e.g., Chicago Illinois)",
    "Date matched from the weather dataset (YYYY-MM-DD)",
    "Maximum temperature recorded on that day (°F)",
    "Minimum temperature recorded on that day (°F)",
    "Average daily temperature (°F)",
    "'Feels Like' temperature (°F)",
    "Humidity percentage on that day",
    "Precipitation measured in inches",
    "Snowfall recorded that day in inches",
    "Depth of snow on the ground (inches)",
    "Average wind speed in miles per hour",
    "Percentage of sky covered by clouds",
    "Visibility distance in miles",
    "Ultraviolet index score (0–11+)",
    "Risk level for severe weather (0–100 scale)",
    "General weather description (e.g., 'Rain', 'Cloudy')"
]


updated_dict = pd.DataFrame({
    "Column Name": column_names,
    "Description": descriptions
})

display(updated_dict)

,Column Name,Description
0,Rating,"Star rating in text form (e.g., '4.3 stars')"
1,Clean_Date,Cleaned review date in YYYY-MM-DD format
2,Location,Restaurant or cafe name where review was left
3,NumericRating,Numeric version of star rating (float)
4,City,"City and state of the location (e.g., Chicago ..."
5,Date_y,Date matched from the weather dataset (YYYY-MM...
6,Temp Max (°F),Maximum temperature recorded on that day (°F)
7,Temp Min (°F),Minimum temperature recorded on that day (°F)
8,Avg Temp (°F),Average daily temperature (°F)
9,Feels Like (°F),'Feels Like' temperature (°F)
